# Working with Rows

## Import Libraries

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

## SparkSession

In [2]:
spark = (SparkSession
    .builder
    .master("local")
    .config(conf=SparkConf())
    .getOrCreate()
)

In [3]:
# Display settings
from IPython.core.display import HTML
display(HTML("<style>pre {white-space: pre !important; }</style>"))

## Load the data

In [4]:
import os
from pyspark.sql.functions import to_timestamp, col, lit

data_path = 'file:///' + os.getcwd() + '/data'

file_path = data_path + '/reported-crimes.csv'

crimes_df = (spark.read
        .csv(file_path, header=True)
        .withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a'))
        .filter(col('Date') <= lit('2018-11-11'))
)
crimes_df.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

**Add the reported crimes for an additional day, 12-Nov-2018, to our dataset**

In [5]:
one_day = (spark.read
        .csv(file_path, header=True)
        .withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a'))
        .filter(col('Date') == lit('2018-11-12'))
)
one_day.count()

3

In [6]:
crimes_df.union(one_day).orderBy('Date', ascending=False).show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00|  003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| fal

**What are the top 10 number of crimes by Primary type, in descending order of occurence?**

In [7]:
(crimes_df
     .groupBy('Primary Type')
     .count().
     orderBy('count', ascending=False).show(10))

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1418467|
|            BATTERY|1232249|
|    CRIMINAL DAMAGE| 771505|
|          NARCOTICS| 711748|
|      OTHER OFFENSE| 418877|
|            ASSAULT| 418512|
|           BURGLARY| 388038|
|MOTOR VEHICLE THEFT| 314130|
| DECEPTIVE PRACTICE| 266487|
|            ROBBERY| 255598|
+-------------------+-------+
only showing top 10 rows



**What percentage of reported crimes resulted in an arrest?**

In [8]:
crimes_df.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [9]:
crimes_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [10]:
arrests = crimes_df.filter(col('Arrest') == 'true').count() 
total_reported = crimes_df.select('Arrest').count()
percentage_arrests = arrests / total_reported * 100
print('Percentage arrests: {}'.format(percentage_arrests))

Percentage arrests: 27.754665335437362


**What are the top 3 locations for reported crimes**

In [11]:
crimes_df.groupBy('Location Description').count().orderBy('count', ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770591|
|           RESIDENCE|1145442|
|           APARTMENT| 698632|
+--------------------+-------+
only showing top 3 rows

